In [ ]:
# !sudo -H pip3 install lightgbm --install-option=--gpu

In [ ]:
# !pip3 install pandas
# !pip3 install numpy
# !pip3 install scipy
# !pip3 install scikit-learn
# !pip3 install xgboost
# !pip3 install lightgbm

In [11]:
import os
import gc
import sys
sys.path.append("../..")
import time

import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
import lightgbm as lgb

from Other.utils.ReduceMem import read_and_reduce, merge_and_reduce

In [4]:
def check_and_drop_column(df, column):
    if column in df.columns:
        df.drop(columns=[column], axis=1, inplace=True)

In [5]:
def print_columns(df):
    s = ''
    for col in df.columns:
        s += '"%s", ' % col
    print(s[:-2])

In [42]:
def start_time():
    # Set timezone
    os.environ['TZ'] = 'Asia/Seoul'
    time.tzset()
    
    # Print current iteration information
    start = time.time()
    now = time.localtime(start)
    print('Started..., Time: {yy}-{mm}-{dd} {hh}:{MM}:{ss}'.format(yy=now.tm_year, mm=now.tm_mon, dd=now.tm_mday, hh=now.tm_hour, MM=now.tm_min, ss=now.tm_sec))

In [37]:
def end_time():
    # Measure time elapsed
    now = time.time()
    time_elapsed = now - start
    print('Completed, Time elapsed: {} seconds'.format(time_elapsed))

# Preprocessing

In [8]:
pwd

'/home/etture2/ybigta_first_proj/Models/3_combined_model_3'

In [9]:
train = read_and_reduce('../../../Data/derived/train.csv')
test = read_and_reduce('../../../Data/derived/test.csv')

Mem. usage decreased to 596.41 Mb (64.4% reduction)
Mem. usage decreased to 518.98 Mb (63.8% reduction)


In [10]:
print(train.shape)
print(test.shape)

(590540, 372)
(506691, 371)


# LightGBM

In [19]:
!nvidia-smi

Thu Aug 22 14:55:39 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.00    Driver Version: 418.87.00    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P0    72W / 149W |   1006MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [12]:
y_train = train['isFraud'].copy()

In [13]:
X_train = train.drop('isFraud', axis=1)
X_test = test.copy()

In [14]:
# Fill numeric columns' NaN values with mean
for c in X_train.columns:
    if X_train[c].dtype=='float16' or  X_train[c].dtype=='float32' or  X_train[c].dtype=='float64':
        X_train[c].fillna(X_train[c].mean())
        X_test[c].fillna(X_train[c].mean())

In [15]:
# Fill categorical columns' NaN values with placeholder (-999)
X_train = X_train.fillna(-999)
X_test = X_test.fillna(-999)

In [16]:
# Label-Encode categorical columns
for f in X_train.columns:
    if X_train[f].dtype=='object' or X_test[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f].values) + list(X_test[f].values))
        X_train[f] = lbl.transform(list(X_train[f].values))
        X_test[f] = lbl.transform(list(X_test[f].values))

In [17]:
# Set up K-Fold
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True)

print(folds)

KFold(n_splits=5, random_state=None, shuffle=True)


In [19]:
# Initialize submission DataFrame
sample_submission = pd.read_csv('../../../Data/original/sample_submission.csv')
lgb_submission = sample_submission.copy()
lgb_submission['isFraud'] = 0


In [21]:
np.arange(400, 2000, 300)

array([ 400,  700, 1000, 1300, 1600, 1900])

## GridSearch LightGBM hyperparameters

In [ ]:
# GridSearch
gridParams = {
    'learning_rate': np.arange(0.02, 0.06, 0.01),
    'num_leaves': [2**5, 2**6, 2**7, 2**8],
    'max_depth': [8, 10]
}

lgbm_classifier = lgb.LGBMClassifier(
    boosting_type='gbdt',
    objective='binary',
    n_jobs=-1,
    silent=True,
    subsample_freq=1
)

print('-------------- default model params')
print(lgbm_classifier.get_params().keys())

# Print current iteration information
start = time.time()
now = time.localtime(start)
print('GridSearch started..., Time: {yy}-{mm}-{dd} {hh}:{MM}:{ss}'.format(yy=now.tm_year, mm=now.tm_mon, dd=now.tm_mday, hh=now.tm_hour, MM=now.tm_min, ss=now.tm_sec))

grid = GridSearchCV(lgbm_classifier, gridParams, verbose=100, cv=4, n_jobs=16)
grid.fit(X_train, y_train)

# Measure time elapsed
now = time.time()
time_elapsed = now - start
print('Completed, Time elapsed: {} seconds'.format(time_elapsed))

print('best params=======')
print(grid.best_params_)
print('best score========')
print(grid.best_score_)


-------------- default model params
dict_keys(['boosting_type', 'class_weight', 'colsample_bytree', 'importance_type', 'learning_rate', 'max_depth', 'min_child_samples', 'min_child_weight', 'min_split_gain', 'n_estimators', 'n_jobs', 'num_leaves', 'objective', 'random_state', 'reg_alpha', 'reg_lambda', 'silent', 'subsample', 'subsample_for_bin', 'subsample_freq'])
GridSearch started..., Time: 2019-8-22 19:51:32
Fitting 4 folds for each of 32 candidates, totalling 128 fits
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
Pickling array (shape=(371,), dtype=object).
Memmapping (shape=(280, 590540), dtype=float16) to new file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-dbd7fc42742345359ca81afc3506df1a.pkl
Memmapping (shape=(44, 590540), dtype=float32) to new file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-d125e6f7d0754a81963be336373e6e70.pkl
Memmapping (shape=(1, 590540), dtype=float64) to new file /dev/shm

Memmapping (shape=(1, 590540), dtype=int64) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-94406d1ac34c423c98c6dd66a33ffddd.pkl
Memmapping (shape=(1, 590540), dtype=int64) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-611157a45db4490b8b6216bc3719b2bc.pkl
Memmapping (shape=(1, 590540), dtype=int64) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-a1cbe7ccf38641a3a1fb0421c6249281.pkl
Memmapping (shape=(1, 590540), dtype=int64) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-5bd8460dcef1445da693507feeabe131.pkl
Memmapping (shape=(1, 590540), dtype=int64) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-840f92f9472a4c68a59b21d87e9b1bb6.pkl
Memmapping (shape=(1, 590540), dtype=int64) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-7ac644b352024254830c742cdabcc196.pkl
Memmapping (shap

Pickling array (shape=(590540,), dtype=int8).
Memmapping (shape=(442904,), dtype=int64) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-18310395d543498a9678b5df98a50553.pkl
Memmapping (shape=(147636,), dtype=int64) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-3899fc985f6e4be39812a04adf021510.pkl
Pickling array (shape=(371,), dtype=object).
Memmapping (shape=(280, 590540), dtype=float16) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-dbd7fc42742345359ca81afc3506df1a.pkl
Memmapping (shape=(44, 590540), dtype=float32) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-d125e6f7d0754a81963be336373e6e70.pkl
Memmapping (shape=(1, 590540), dtype=float64) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-4f0421ece8344b4291014ddb32126826.pkl
Memmapping (shape=(3, 590540), dtype=int16) to old file /dev/shm/joblib_memmapping_f

[Parallel(n_jobs=16)]: Done   1 tasks      | elapsed:  3.6min
Pickling array (shape=(371,), dtype=object).
Memmapping (shape=(280, 590540), dtype=float16) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-dbd7fc42742345359ca81afc3506df1a.pkl
Memmapping (shape=(44, 590540), dtype=float32) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-d125e6f7d0754a81963be336373e6e70.pkl
Memmapping (shape=(1, 590540), dtype=float64) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-4f0421ece8344b4291014ddb32126826.pkl
Memmapping (shape=(3, 590540), dtype=int16) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-762ba7c18063462eab7c1df57516c12c.pkl
Memmapping (shape=(9, 590540), dtype=int32) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-36d562c5b49042aaab1c914d7f552565.pkl
Memmapping (shape=(1, 590540), dtype=int64) to old file /dev/shm

[Parallel(n_jobs=16)]: Done   2 tasks      | elapsed:  3.6min
Pickling array (shape=(371,), dtype=object).
Memmapping (shape=(280, 590540), dtype=float16) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-dbd7fc42742345359ca81afc3506df1a.pkl
Memmapping (shape=(44, 590540), dtype=float32) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-d125e6f7d0754a81963be336373e6e70.pkl
Memmapping (shape=(1, 590540), dtype=float64) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-4f0421ece8344b4291014ddb32126826.pkl
Memmapping (shape=(3, 590540), dtype=int16) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-762ba7c18063462eab7c1df57516c12c.pkl
Memmapping (shape=(9, 590540), dtype=int32) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-36d562c5b49042aaab1c914d7f552565.pkl
Memmapping (shape=(1, 590540), dtype=int64) to old file /dev/shm

...

[Parallel(n_jobs=16)]: Done  77 tasks      | elapsed: 21.9min
Pickling array (shape=(371,), dtype=object).
Memmapping (shape=(280, 590540), dtype=float16) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-dbd7fc42742345359ca81afc3506df1a.pkl
Memmapping (shape=(44, 590540), dtype=float32) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-d125e6f7d0754a81963be336373e6e70.pkl
Memmapping (shape=(1, 590540), dtype=float64) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-4f0421ece8344b4291014ddb32126826.pkl
Memmapping (shape=(3, 590540), dtype=int16) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-762ba7c18063462eab7c1df57516c12c.pkl
Memmapping (shape=(9, 590540), dtype=int32) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-36d562c5b49042aaab1c914d7f552565.pkl
Memmapping (shape=(1, 590540), dtype=int64) to old file /dev/shm

[Parallel(n_jobs=16)]: Done  78 tasks      | elapsed: 22.0min
Pickling array (shape=(371,), dtype=object).
Memmapping (shape=(280, 590540), dtype=float16) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-dbd7fc42742345359ca81afc3506df1a.pkl
Memmapping (shape=(44, 590540), dtype=float32) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-d125e6f7d0754a81963be336373e6e70.pkl
Memmapping (shape=(1, 590540), dtype=float64) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-4f0421ece8344b4291014ddb32126826.pkl
Memmapping (shape=(3, 590540), dtype=int16) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-762ba7c18063462eab7c1df57516c12c.pkl
Memmapping (shape=(9, 590540), dtype=int32) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-36d562c5b49042aaab1c914d7f552565.pkl
Memmapping (shape=(1, 590540), dtype=int64) to old file /dev/shm

[Parallel(n_jobs=16)]: Done  79 tasks      | elapsed: 22.2min
Pickling array (shape=(371,), dtype=object).
Memmapping (shape=(280, 590540), dtype=float16) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-dbd7fc42742345359ca81afc3506df1a.pkl
Memmapping (shape=(44, 590540), dtype=float32) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-d125e6f7d0754a81963be336373e6e70.pkl
Memmapping (shape=(1, 590540), dtype=float64) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-4f0421ece8344b4291014ddb32126826.pkl
Memmapping (shape=(3, 590540), dtype=int16) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-762ba7c18063462eab7c1df57516c12c.pkl
Memmapping (shape=(9, 590540), dtype=int32) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-36d562c5b49042aaab1c914d7f552565.pkl
Memmapping (shape=(1, 590540), dtype=int64) to old file /dev/shm

Memmapping (shape=(147634,), dtype=int64) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-3cc4f6d94fb546e1a125c592b4db2ab5.pkl
[Parallel(n_jobs=16)]: Done  80 tasks      | elapsed: 22.2min
Pickling array (shape=(371,), dtype=object).
Memmapping (shape=(280, 590540), dtype=float16) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-dbd7fc42742345359ca81afc3506df1a.pkl
Memmapping (shape=(44, 590540), dtype=float32) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-d125e6f7d0754a81963be336373e6e70.pkl
Memmapping (shape=(1, 590540), dtype=float64) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-4f0421ece8344b4291014ddb32126826.pkl
Memmapping (shape=(3, 590540), dtype=int16) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-762ba7c18063462eab7c1df57516c12c.pkl
Memmapping (shape=(9, 590540), dtype=int32) to old file /dev/shm/j

[Parallel(n_jobs=16)]: Done  81 tasks      | elapsed: 22.3min
Pickling array (shape=(371,), dtype=object).
Memmapping (shape=(280, 590540), dtype=float16) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-dbd7fc42742345359ca81afc3506df1a.pkl
Memmapping (shape=(44, 590540), dtype=float32) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-d125e6f7d0754a81963be336373e6e70.pkl
Memmapping (shape=(1, 590540), dtype=float64) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-4f0421ece8344b4291014ddb32126826.pkl
Memmapping (shape=(3, 590540), dtype=int16) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-762ba7c18063462eab7c1df57516c12c.pkl
Memmapping (shape=(9, 590540), dtype=int32) to old file /dev/shm/joblib_memmapping_folder_9503_8435464038/9503-140487994816888-36d562c5b49042aaab1c914d7f552565.pkl
Memmapping (shape=(1, 590540), dtype=int64) to old file /dev/shm

### GridSearch Results

#### Test parameters

```python
gridParams = {
    'learning_rate': np.arange(0.02, 0.06, 0.01),
    'num_leaves': [2**5, 2**6, 2**7, 2**8],
    'max_depth': [8, 10]
}
```

#### best parameters

```python
{'learning_rate': 0.02, 'max_depth': 8, 'num_leaves': 128}
```

#### best score

```
0.9690046398211806
```

In [ ]:
gridParams = {
    'colsample_bytree': np.arange(0.7, 0.9, 0.1),
    'subsample': np.arange(0.7, 0.9, 0.1),
    'reg_alpha': np.arange(0.2, 1.1, 0.25),
    'reg_lambda': np.arange(0.2, 0.5, 0.1)
}

lgbm_classifier = lgb.LGBMClassifier(
    boosting_type='gbdt',
    objective='binary',
    n_jobs=-1,
    silent=True,
    subsample_freq=1
)

print('-------------- default model params')
print(lgbm_classifier.get_params().keys())

# Print current iteration information
start = time.time()
now = time.localtime(start)
print('GridSearch started..., Time: {yy}-{mm}-{dd} {hh}:{MM}:{ss}'.format(yy=now.tm_year, mm=now.tm_mon, dd=now.tm_mday, hh=now.tm_hour, MM=now.tm_min, ss=now.tm_sec))

grid = GridSearchCV(lgbm_classifier, gridParams, verbose=100, cv=4, n_jobs=8)
grid.fit(X_train, y_train)

# Measure time elapsed
now = time.time()
time_elapsed = now - start
print('Completed, Time elapsed: {} seconds'.format(time_elapsed))

print('best params=======')
print(grid.best_params_)
print('best score========')
print(grid.best_score_)

### GridSearch Results

#### Test parameters

```python
gridParams = {
    'colsample_by_tree': np.arange(0.7, 0.9, 0.1),
    'subsample': np.arange(0.7, 0.9, 0.1),
    'reg_alpha': np.arange(0.2, 1.1, 0.25),
    'reg_lambda': np.arange(0.2, 0.5, 0.1)
}
```

#### best params

```python
{'colsample_by_tree': 0.7, 'reg_alpha': 0.45, 'reg_lambda': 0.4000000000000001, 'subsample': 0.8999999999999999}
```

#### best score
```
0.8808277170047752
```

In [32]:
# If using GPU, set this option to True
gpu_enabled = False

params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'n_estimators': 2000,
    'colsample_bytree': 0.85,
    'subsample_freq': 1,
    'subsample': 0.85,
    'max_bin': 255,
    'metric': 'auc',
    'n_jobs': -1,
    'num_leaves': 2**8,
    'max_depth': 10,
    'tree_learner': 'serial',
    'learning_rate': 0.05,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'reg_alpha': 0.3,
    'reg_lambda': 0.243
}

if gpu_enabled:
    params.update({
        'device': 'gpu',
        'gpu_platform_id': 0,
        'gpu_device_id': 0
    })
    
params['learning_rate'] = 0.02
params['max_depth'] = 8
params['num_leaves'] = 128
params['colsample_bytree'] = 0.7
params['subsample'] = 0.9
params['reg_alpha'] = 0.45
params['reg_lambda'] = 0.4

for fold_n, (train_index, valid_index) in enumerate(folds.split(X_train)):    
    # Print current iteration information
    start = time.time()
    now = time.localtime(start)
    print('Fold {} started..., Time: {yy}-{mm}-{dd} {hh}:{MM}:{ss}'.format(fold_n, yy=now.tm_year, mm=now.tm_mon, dd=now.tm_mday, hh=now.tm_hour, MM=now.tm_min, ss=now.tm_sec))
    
    X_train_, X_valid = X_train.iloc[train_index], X_train.iloc[valid_index]
    y_train_, y_valid = y_train.iloc[train_index], y_train.iloc[valid_index]
    
    lgb_train = lgb.Dataset(X_train_, label=y_train_)
    lgb_test = lgb.Dataset(X_valid, label=y_valid)
    
    lgb_estimator = lgb.train(
        params,
        lgb_train,
        valid_sets = [lgb_test],
        verbose_eval=100,
        early_stopping_rounds=50
    )
    
    del X_train_, y_train_
    
    pred = lgb_estimator.predict(X_test)
    val = lgb_estimator.predict(X_valid)
    
    # Measure time elapsed
    now = time.time()
    time_elapsed = now - start
    print('Completed, Time elapsed: {} seconds'.format(time_elapsed))
    
    print('ROC accuracy: {}'.format(roc_auc_score(y_valid, val)))
    
    del val, y_valid
    
    lgb_submission['isFraud'] = lgb_submission['isFraud'] + pred/n_fold
    
    del pred
    gc.collect()
    
    print('-------------------------------------------------------')
    
print('Done.')

Fold 0 started..., Time: 2019-8-22 22:32:13


/home/etture2/.local/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.91177
[200]	valid_0's auc: 0.930389
[300]	valid_0's auc: 0.938812
[400]	valid_0's auc: 0.944493
[500]	valid_0's auc: 0.948388
[600]	valid_0's auc: 0.951512
[700]	valid_0's auc: 0.954485
[800]	valid_0's auc: 0.956921
[900]	valid_0's auc: 0.958721
[1000]	valid_0's auc: 0.960556
[1100]	valid_0's auc: 0.962358
[1200]	valid_0's auc: 0.96376
[1300]	valid_0's auc: 0.965303
[1400]	valid_0's auc: 0.966711
[1500]	valid_0's auc: 0.967588
[1600]	valid_0's auc: 0.968354
[1700]	valid_0's auc: 0.969057
[1800]	valid_0's auc: 0.969751
[1900]	valid_0's auc: 0.970393
[2000]	valid_0's auc: 0.970968
Did not meet early stopping. Best iteration is:
[1999]	valid_0's auc: 0.97097
Completed, Time elapsed: 238.37578773498535 seconds
ROC accuracy: 0.9709697726556069
-------------------------------------------------------
Fold 1 started..., Time: 2019-8-22 22:36:12


/home/etture2/.local/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.909962
[200]	valid_0's auc: 0.927701
[300]	valid_0's auc: 0.936482
[400]	valid_0's auc: 0.942227
[500]	valid_0's auc: 0.946061
[600]	valid_0's auc: 0.949122
[700]	valid_0's auc: 0.952574
[800]	valid_0's auc: 0.954919
[900]	valid_0's auc: 0.957725
[1000]	valid_0's auc: 0.959953
[1100]	valid_0's auc: 0.961773
[1200]	valid_0's auc: 0.963419
[1300]	valid_0's auc: 0.964819
[1400]	valid_0's auc: 0.965751
[1500]	valid_0's auc: 0.967004
[1600]	valid_0's auc: 0.968003
[1700]	valid_0's auc: 0.969059
[1800]	valid_0's auc: 0.96982
[1900]	valid_0's auc: 0.970539
[2000]	valid_0's auc: 0.971408
Did not meet early stopping. Best iteration is:
[2000]	valid_0's auc: 0.971408
Completed, Time elapsed: 235.15939283370972 seconds
ROC accuracy: 0.97140779489516
-------------------------------------------------------
Fold 2 started..., Time: 2019-8-22 22:40:8


...

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.907862
[200]	valid_0's auc: 0.926284
[300]	valid_0's auc: 0.935816
[400]	valid_0's auc: 0.940678
[500]	valid_0's auc: 0.945399
[600]	valid_0's auc: 0.949164
[700]	valid_0's auc: 0.951587
[800]	valid_0's auc: 0.954127
[900]	valid_0's auc: 0.956578
[1000]	valid_0's auc: 0.958646
[1100]	valid_0's auc: 0.960427
[1200]	valid_0's auc: 0.961745
[1300]	valid_0's auc: 0.962917
[1400]	valid_0's auc: 0.963976
[1500]	valid_0's auc: 0.965077
[1600]	valid_0's auc: 0.9661
[1700]	valid_0's auc: 0.96693
[1800]	valid_0's auc: 0.967843
[1900]	valid_0's auc: 0.968733
[2000]	valid_0's auc: 0.969391
Did not meet early stopping. Best iteration is:
[2000]	valid_0's auc: 0.969391
Completed, Time elapsed: 239.8993799686432 seconds
ROC accuracy: 0.9693905750532483
-------------------------------------------------------
Fold 3 started..., Time: 2019-8-22 22:44:8


/home/etture2/.local/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.9101
[200]	valid_0's auc: 0.93022
[300]	valid_0's auc: 0.938051
[400]	valid_0's auc: 0.943405
[500]	valid_0's auc: 0.947676
[600]	valid_0's auc: 0.951047
[700]	valid_0's auc: 0.95404
[800]	valid_0's auc: 0.956359
[900]	valid_0's auc: 0.959043
[1000]	valid_0's auc: 0.960474
[1100]	valid_0's auc: 0.962029
[1200]	valid_0's auc: 0.963765
[1300]	valid_0's auc: 0.965075
[1400]	valid_0's auc: 0.9663
[1500]	valid_0's auc: 0.967439
[1600]	valid_0's auc: 0.968245
[1700]	valid_0's auc: 0.969044
[1800]	valid_0's auc: 0.969656
[1900]	valid_0's auc: 0.970436
[2000]	valid_0's auc: 0.971053
Did not meet early stopping. Best iteration is:
[2000]	valid_0's auc: 0.971053
Completed, Time elapsed: 239.45052218437195 seconds
ROC accuracy: 0.9710528314147993
-------------------------------------------------------
Fold 4 started..., Time: 2019-8-22 22:48:7


/home/etture2/.local/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.908274
[200]	valid_0's auc: 0.927048
[300]	valid_0's auc: 0.935067
[400]	valid_0's auc: 0.940517
[500]	valid_0's auc: 0.944746
[600]	valid_0's auc: 0.948134
[700]	valid_0's auc: 0.951215
[800]	valid_0's auc: 0.953783
[900]	valid_0's auc: 0.955843
[1000]	valid_0's auc: 0.958442
[1100]	valid_0's auc: 0.960333
[1200]	valid_0's auc: 0.961967
[1300]	valid_0's auc: 0.963072
[1400]	valid_0's auc: 0.964152
[1500]	valid_0's auc: 0.965139
[1600]	valid_0's auc: 0.966262
[1700]	valid_0's auc: 0.967109
[1800]	valid_0's auc: 0.967951
[1900]	valid_0's auc: 0.968642
[2000]	valid_0's auc: 0.969363
Did not meet early stopping. Best iteration is:
[2000]	valid_0's auc: 0.969363
Completed, Time elapsed: 236.46723175048828 seconds
ROC accuracy: 0.9693626696659339
-------------------------------------------------------
Done.


In [34]:
dir_name = 'submission'
lgb_submission.to_csv('./{directory}/sub_lightgbm_cpu_2.csv'.format(directory=dir_name), index=False)
lgb_submission.head()

,TransactionID,isFraud
0,3663549,0.001002
1,3663550,0.000894
2,3663551,0.000455
3,3663552,0.000936
4,3663553,0.000869


# DONE

#### Kaggle submission score: 0.9430
##### Previous score: 0.9421, advanced 186 places on leaderboard

# RFECV

In [35]:
from sklearn.feature_selection import RFECV

In [68]:
infs = np.where(X_train.values >= np.finfo(np.float64).max)

In [70]:
len(infs[1])

1448

In [75]:
X_train_ = X_train.copy()

X_train_[X_train_==np.inf] = np.nan
X_train_.fillna(X_train_.mean(), inplace=True)
# for idx, inf in enumerate(infs[0]):
#     X_train_.iloc[inf[0][idx], inf[1][idx]] = 

In [76]:
infs = np.where(X_train_.values >= np.finfo(np.float64).max)
len(infs[0])

0

In [83]:
folds = 5
step = 2

params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'n_estimators': 2000,
    'colsample_bytree': 0.7,
    'subsample_freq': 1,
    'subsample': 0.9,
    'max_bin': 255,
    'metric': 'auc',
    'n_jobs': -1,
    'num_leaves': 2**6,
    'max_depth': 7,
    'tree_learner': 'serial',
    'learning_rate': 0.02,
    'verbose': 0,
    'reg_alpha': 0.45,
    'reg_lambda': 0.4,
    'boost_from_average': False
}

lgbm_classifier = lgb.LGBMClassifier(**params)

rfecv = RFECV(
    estimator=lgbm_classifier,
    step=step,
    cv=KFold(n_splits=folds, shuffle=False),
    scoring='roc_auc',
    n_jobs=16,
    verbose=2
)

# lgb_train = lgb.Dataset(X_train_, label=y_train_)
# lgb_test = lgb.Dataset(X_valid, label=y_valid)

# X_train_ = X_train.fillna(X_train.mean())
# y_train_ = y_train.reset_index()

start_time()
rfecv.fit(X_train_, y_train)
end_time()

all_features = [x for x in X_train.columns]

print('\n Optimal number of features: %d' % rfecv.n_features_)
sel_features = [f for f, s in zip(all_features, rfecv.support_) if s]
print('\n The selected features are {}:'.format(sel_features))




Started..., Time: 2019-8-23 1:12:42


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker. The exit codes of the workers are {SIGSEGV(-11)}

In [ ]:
plt.figure(figsize=(12, 9))
plt.xlabel('Number of features tested x 2')
plt.ylabel('Cross-validation score (AUC)')
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.savefig('lightgbm-RFECV-01.png', dpi=150)
plt.show()